In [1]:
# !pip install opencv-python
# !pip install --upgrade pandas
# !pip install --upgrade scipy==1.0.0
# !pip install --upgrade matplotlib==2.1.2
# !pip install --upgrade pyarrow==0.8.0
# ! pip install --upgrade bcolz
# %cd fastai
# !python setup.py install
# !wget http://files.fast.ai/models/weights.tgz /output/weights.tgz
# !tar -xvzf /prev_output/weights.tgz -C /usr/local/lib/python3.6/site-packages/fastai-0.6-py3.6.egg/fastai/ 
# %cp -r /data /output/data

In [2]:
!ls /prev_output

Untitled.ipynb	data  fastai  run_cpu.sh  weights.tgz


In [3]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *


In [5]:
PATH = "/output/data/"
arch=resnext101_64
sz=224
bs = 64

In [6]:
!ls {PATH}

data  labels.csv  raw  sample  sample_submission.csv  test


In [7]:
tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
data = ImageClassifierData.from_paths(PATH+'raw', tfms=tfms_from_model(arch, sz), trn_name='trn', test_name=PATH+'test/test_raw')


In [8]:
len(data.trn_ds), len(data.test_ds)

(9403, 10357)

In [9]:
len(data.classes), data.classes[:5]

(120,
 ['affenpinscher',
  'afghan_hound',
  'african_hunting_dog',
  'airedale',
  'american_staffordshire_terrier'])

In [10]:
def get_data(sz, bs):
    tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
    data = ImageClassifierData.from_paths(PATH+'raw', tfms=tfms_from_model(arch, sz), trn_name='trn', test_name=PATH+'test/test_raw')
    return data if sz>300 else data.resize(340, 'tmp')

In [11]:
data = get_data(sz, bs)

A Jupyter Widget

In [12]:
learn = ConvLearner.pretrained(arch, data, precompute=True)

100%|██████████| 162/162 [05:10<00:00,  1.91s/it]


In [13]:
learn.fit(1e-2, 5)

A Jupyter Widget

epoch      trn_loss   val_loss   accuracy                    
    0      0.876556   0.33468    0.936864  
    1      0.403694   0.253087   0.940163                    
    2      0.29721    0.237245   0.937759                    
    3      0.231263   0.216359   0.932952                    
    4      0.190859   0.20644    0.94318                     



[0.20644008, 0.94317967616594756]

In [15]:
learn.precompute=False
learn.fit(1e-2, 5, cycle_len=1)

A Jupyter Widget

epoch      trn_loss   val_loss   accuracy                    
    0      0.156458   0.202692   0.939268  
    1      0.14217    0.201379   0.937759                    
    2      0.132878   0.200384   0.941082                    
    3      0.112125   0.19811    0.941978                    
    4      0.106259   0.203871   0.942873                    



[0.20387146, 0.94287330370682931]

In [16]:
learn.save('/output/244_pre')

In [19]:
test = learn.predict(data.test_dl)

KeyboardInterrupt: 

In [20]:
log_preds = learn.predict()

In [22]:
log_preds.shape

(819, 120)

In [23]:
log_preds_test = learn.predict_dl(data.test_dl)

In [24]:
log_preds_test.shape

(10357, 120)

In [31]:
import pickle as pkl
pkl.dump(log_preds_test, open('/output/result.pkl', 'wb'))

In [29]:
!ls /output/

244_pre.h5  fastai		    pg.ipynb	run_gpu.sh
data	    floyd_requirements.txt  run_cpu.sh


In [32]:
def do_clip(arr, mx): return np.clip(arr, (1-mx)/9, mx)


In [ ]:
import keras
keras.metrics.categorical_crossentropy(data.val_y, do_clip(log_preds, 0.93)).eval()


In [35]:
!pip install tensorflow

    100% |████████████████████████████████| 44.4MB 34kB/s  eta 0:00:01
    100% |████████████████████████████████| 3.0MB 501kB/s eta 0:00:01
    100% |████████████████████████████████| 6.4MB 239kB/s eta 0:00:01
    100% |████████████████████████████████| 81kB 10.8MB/s ta 0:00:01
    100% |████████████████████████████████| 890kB 1.6MB/s eta 0:00:01
    100% |████████████████████████████████| 327kB 3.8MB/s eta 0:00:01
    100% |████████████████████████████████| 81kB 10.7MB/s ta 0:00:01
  Running setup.py bdist_wheel for absl-py ... done
  Stored in directory: /root/.cache/pip/wheels/04/f5/7c/5d4eab10ddf87dec875016e74ba289d87270a90fb2662a76fc
  Running setup.py bdist_wheel for futures ... done
  Stored in directory: /root/.cache/pip/wheels/ad/79/48/b32521764d59b16fd1bc0ffd5862f6d3bf770c7d73ea1fb12a
  Running setup.py bdist_wheel for html5lib ... done
  Stored in directory: /root/.cache/pip/wheels/6f/85/6c/56b8e1292c6214c4eb73b9dda50f53e8e977bf65989373c962
Successfully built absl-py future